# Tutorial for trXPS for the HEXTOF instrument at FLASH: t0, cross-correlation and BAM correction

## Preparation

### Import necessary libraries

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os

from sed import SedProcessor
from sed.dataset import dataset
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

# For peak fitting
from lmfit.models import GaussianModel

### Get data paths

If it is your beamtime, you can read the raw data and write to the processed directory. For the public data, you can not write to the processed directory.

The paths are such that if you are on Maxwell, it uses those. Otherwise, data is downloaded in the current directory from Zenodo:
https://zenodo.org/records/12609441

In [ ]:
beamtime_dir = "/asap3/flash/gpfs/pg2/2023/data/11019101" # on Maxwell
if os.path.exists(beamtime_dir) and os.access(beamtime_dir, os.R_OK):
    path = beamtime_dir + "/raw/hdf/offline/fl1user3"
    buffer_path = beamtime_dir + "/processed/tutorial/"
else:
    # data_path can be defined and used to store the data in a specific location
    dataset.get("W110") # Put in Path to a storage of at least 10 Byte free space.
    path = dataset.dir
    buffer_path = path + "/processed/"

### Config setup
Here, we get the path to the config file and set up the relevant directories. This can also be done directly in the config file.

In [ ]:
# pick the default configuration file for hextof@FLASH
config_file = Path('../sed/config/flash_example_config.yaml')
assert config_file.exists()

In [ ]:
# here we setup a dictionary that will be used to override the path configuration
config_override = {
    "core": {
        "beamtime_id": 11019101,
        "paths": {
            "data_raw_dir": path,
            "data_parquet_dir": buffer_path
        },
    },
}

In [ ]:
energy_cal = {
    "energy": {
        "calibration": {
            "E0": -53.96145014592986,
            "creation_date": 1732056868.029444,
            "d": 0.8096677233434938,
            "energy_scale": "kinetic",
            "t0": 4.0148196718030886e-07,
        },
        "offsets":{
            "constant": -77.5,
            "creation_date": 1732056874.060922,
            "monochromatorPhotonEnergy": {
                "preserve_mean": True,
                "weight": -1,
            },
            "sampleBias": {
                "preserve_mean": False,
                "weight": 1,
            },
            "tofVoltage": {
                "preserve_mean": True,
                "weight": -1,
            },
        },
    },
}

### We use the stored energy calibration parameters and load trXPS data set to define:
* t0 position with respect to delay stage values;
* correct accordingly delay stage offset
* fit cross-correlation 
* apply BAM correction and see its effect on cross-correlation

In [ ]:
run_number = 44498
sp_44498 = SedProcessor(runs=[run_number], config=config_override, folder_config=energy_cal, system_config=config_file, verbose=True)

sp_44498.add_jitter()
sp_44498.align_dld_sectors()
sp_44498.append_energy_axis()
sp_44498.add_energy_offset()

Check which channels are included in the dataframe

In [ ]:
sp_44498.dataframe.head()

## Data w/o BAM correction

First, we take a look at our sideband measurement before any corrections.
The sidebands on the W4f core levels can be used as a measure of the pump and probe cross-correlation,
and hence our temporal resolution.
We plot the data delay stage position vs Energy data, normalized by acquisition time.

In [ ]:
axes = ['energy', 'delayStage']
ranges = [[-37.5,-27.5], [1446.75,1449.15]]
bins = [200,40]
res = sp_44498.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,3), layout='constrained')
res.plot(robust=True, ax=ax[0], cmap='terrain')
fig.suptitle(f"Run {run_number}: W 4f, side bands")
ax[0].set_title('raw')
bg = res.sel(delayStage=slice(1448.7,1449.1)).mean('delayStage')
(res.sel(delayStage=slice(1446.8,1449.3))-bg).plot(robust=True, ax=ax[1])
ax[1].set_title('difference')

Now we make fit to determine precise t$_0$ position and cross-correlation using lmfit fit models

In [ ]:
Gauss_mod = GaussianModel()

#first order sideband:
x1=res['delayStage']
y1=res.sel(energy=slice(-30.5,-29.5)).sum('energy')
y1=y1-np.mean(y1.sel(delayStage=slice(1448.7,1449.1)))

pars1 = Gauss_mod.make_params(amplitude=0.1, center=1447.8, sigma=0.02)
out1 = Gauss_mod.fit(y1, pars1, x=x1)

#second order sideband
x2=res['delayStage']
y2=res.sel(energy=slice(-29.5,-28.5)).sum('energy')
y2=y2-np.mean(y2.sel(delayStage=slice(1448.7,1449.1)))

pars2 = Gauss_mod.make_params(amplitude=0.1, center=1447.8, sigma=0.02)
out2 = Gauss_mod.fit(y2, pars2, x=x2)

plt.figure()
plt.plot(x1,y1,'rx', label='$1^{st}$ order sideband')
plt.plot(x1,out1.best_fit,'r', label="FWHM = {:.3f} ps".format(out1.values['fwhm']))
plt.legend(loc="best")
plt.title('run44498, W4f, sidebands comparison')
plt.plot(x2,y2,'bx', label='$2^{nd}$ order sideband')
plt.plot(x2,out2.best_fit,'b', label="FWHM = {:.3f} ps".format(out2.values['fwhm']))
plt.legend(loc="best")
plt.xlabel("delayStage [ps]")
plt.ylabel("Intensity [cts/s]")
plt.show()

As we see the sidebands are quite broad and one of the possible reasons for this could be long or short-term drifts (jitter) of the FEL arrival time with respect to e.g. optical laser or differences in the intra-bunch arrival time. To check and correct for this we can look at beam arrival monitor (BAM). The BAM gives a pulse-resolved measure of the FEL arrival time with respect to a master clock.

## Check BAM versus pulse and train IDs

In [ ]:
axes = ['trainId', 'pulseId', 'bam']
ranges = [[1628022640,1628046700], [0,500], [-6400,100]]
bins = [250, 100, 1000]
res_bam = sp_44498.compute(bins=bins, axes=axes, ranges=ranges)

As we can see, jitter between FEL and pump laser is quite significant withing a pulse train as well as over the whole measurement period.

In [ ]:

fig,ax = plt.subplots(1,2,figsize=(8,3), layout='constrained')
res_bam.sel(bam=slice(-6400,-5100)).sum('trainId').plot(ax=ax[0],robust=True, cmap='terrain')
res_bam.sel(bam=slice(-6400,-5100)).sum('pulseId').plot(ax=ax[1],robust=True, cmap='terrain')
plt.show()

## Apply BAM correction

To correct the SASE jitter, using information from the bam column and to calibrate the pump-probe delay axis, we need to shift the delay stage values to centre the pump-probe-time overlap time zero.

In [ ]:
sp_44498.add_delay_offset(
    constant=-1448, # this is time zero position determined from side band fit
    flip_delay_axis=True, # invert the direction of the delay axis
    columns=['bam'], # use the bam to offset the values
    weights=[-0.001], # bam is in fs, delay in ps
    preserve_mean=True # preserve the mean of the delay axis to keep t0 position
)

### bin in the corrected delay axis

In [ ]:
axes = ['energy', 'delayStage']
ranges = [[-37.5,-27.5], [-1.5,1.5]]
bins = [200,60]
res_corr = sp_44498.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,3), layout='constrained')
fig.suptitle(f"Run {run_number}: W 4f, side bands")
res_corr.plot(robust=True, ax=ax[0], cmap='terrain')
ax[0].set_title('raw')
bg = res_corr.sel(delayStage=slice(-1.3,-1.0)).mean('delayStage')
(res_corr-bg).plot(robust=True, ax=ax[1])
ax[1].set_title('difference')

We clearly see an effect of BAM corrections - side bands are visible much nicer and width became smaller.

In [ ]:
sp_44498.save_delay_offsets()

Now we can repeat fit procedure to determine true cross-correlation value.

In [ ]:
Gauss_mod = GaussianModel()

#first order sideband:
x5=res_corr['delayStage'].sel(delayStage=slice(-1.6,1.5))
y5=res_corr.sel(energy=slice(-30.4,-29.5),delayStage=slice(-1.6,1.5)).sum('energy')
y5=y5-np.mean(y5.sel(delayStage=slice(-1.4,-1.0)))

pars5 = Gauss_mod.make_params(amplitude=0.1, center=0.0, sigma=0.02)
out5 = Gauss_mod.fit(y5, pars5, x=x5)

print(out5.fit_report())

#second order sideband
x6=res_corr['delayStage'].sel(delayStage=slice(-1.6,1.5))
y6=res_corr.sel(energy=slice(-29.5,-27.5),delayStage=slice(-1.6,1.5)).sum('energy')
y6=y6-np.mean(y6.sel(delayStage=slice(-1.4,-1.0)))

pars6 = Gauss_mod.make_params(amplitude=0.1, center=0.0, sigma=0.02)
out6 = Gauss_mod.fit(y6, pars6, x=x6)

print(out6.fit_report())

#comparison plot
plt.figure()
plt.plot(x5,y5,'rx', label='$1^{st}$ order sideband')
plt.plot(x5,out5.best_fit,'r', label="FWHM = {:.3f} ps".format(out5.values['fwhm']))
plt.legend(loc="best")
plt.title('run44498, W4f, sidebands comparison')
plt.plot(x6,y6,'bx', label='$2^{nd}$ order sideband')
plt.plot(x6,out6.best_fit,'b', label="FWHM = {:.3f} ps".format(out6.values['fwhm']))
plt.legend(loc="best")
plt.xlabel("pump probe delay [ps]")
plt.ylabel("Intensity [cts/s]")
plt.show()

## Comparison of the BAM correction effect

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(9,7),layout="constrained")

plt.axes(ax[0,0])
res.plot(cmap='terrain', robust=True)
plt.title("W4f, no bam correction")

plt.axes(ax[0,1])
plt.plot(x1,y1,'rx',label='integrated intensity 1. order')
plt.plot(x1,out1.best_fit,'r',label='1. order fit, FWHM = {:.3f} ps'.format(out1.values['fwhm']))
plt.plot(x2,y2,'bx',label='integrated intensity 2. order')
plt.plot(x2,out2.best_fit,'b',label='2. order fit, FWHM = {:.3f} ps'.format(out2.values['fwhm']))
plt.legend(loc=1) 
plt.title("Sidebands without bam correction")

plt.axes(ax[1,0])
res_corr.sel(delayStage=slice(-1.6,1.5)).plot(robust=True,cmap='terrain')
plt.title("W4f, with bam correction")

plt.axes(ax[1,1])
plt.plot(x5,y5,'rx',label='integrated intensity 1. order')
plt.plot(x5,out5.best_fit,'r',label='1. order fit, FWHM = {:.3f} ps'.format(out5.values['fwhm']))
plt.plot(x6,y6,'bx',label='integrated intensity 2. order')
plt.plot(x6,out6.best_fit,'b',label='2. order fit, FWHM = {:.3f} ps'.format(out6.values['fwhm']))
plt.legend(loc=1)
plt.title("Sidebands with bam correction")

fig.suptitle(f'Run {run_number}: Effect of BAM correction',fontsize='22')